In [1]:
import pandas as pd
import numpy as np

credit_card_data = pd.read_csv('creditcard.csv')


In [2]:
shuffled_data = credit_card_data.sample(frac=1)


In [3]:
one_hot_data = pd.get_dummies(shuffled_data, columns=['Class'])

In [4]:
normalized_data = (one_hot_data - one_hot_data.min()) / (one_hot_data.max() - one_hot_data.min())

In [5]:
df_X = normalized_data.drop(['Class_0', 'Class_1'], axis=1)
df_y = normalized_data[['Class_0', 'Class_1']]

In [6]:
ar_X, ar_y = np.asarray(df_X.values, dtype='float32'), np.asarray(df_y.values, dtype='float32')

In [7]:
train_size = int(0.8 * len(ar_X))
(raw_X_train, raw_y_train) = (ar_X[:train_size], ar_y[:train_size])
(raw_X_test, raw_y_test) = (ar_X[train_size:], ar_y[train_size:])


In [8]:
count_legit, count_fraud = np.unique(credit_card_data['Class'], return_counts=True)[1]
fraud_ratio = float(count_fraud / (count_legit + count_fraud))
print('Percent of fraudulent transactions: ', fraud_ratio)

Percent of fraudulent transactions:  0.001727485630620034


In [9]:
weighting = 1 / fraud_ratio
raw_y_train[:, 1] = raw_y_train[:, 1] * weighting


In [10]:
import tensorflow as tf


C:\Users\avinash.t\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
input_dimensions = ar_X.shape[1]
output_dimensions = ar_y.shape[1]
num_layer_1_cells = 100
num_layer_2_cells = 150

In [12]:
X_train_node = tf.placeholder(tf.float32, [None, input_dimensions], name='X_train')
y_train_node = tf.placeholder(tf.float32, [None, output_dimensions], name='y_train')


In [13]:
X_test_node = tf.constant(raw_X_test, name='X_test')
y_test_node = tf.constant(raw_y_test, name='y_test')

In [14]:
weight_1_node = tf.Variable(tf.zeros([input_dimensions, num_layer_1_cells]), name='weight_1')
biases_1_node = tf.Variable(tf.zeros([num_layer_1_cells]), name='biases_1')

In [15]:
weight_2_node = tf.Variable(tf.zeros([num_layer_1_cells, num_layer_2_cells]), name='weight_2')
biases_2_node = tf.Variable(tf.zeros([num_layer_2_cells]), name='biases_2')


In [16]:
weight_3_node = tf.Variable(tf.zeros([num_layer_2_cells, output_dimensions]), name='weight_3')
biases_3_node = tf.Variable(tf.zeros([output_dimensions]), name='biases_3')


In [17]:
def network(input_tensor):
    layer1 = tf.nn.sigmoid(tf.matmul(input_tensor, weight_1_node) + biases_1_node)
    layer2 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(layer1, weight_2_node) + biases_2_node), 0.85)
    layer3 = tf.nn.softmax(tf.matmul(layer2, weight_3_node) + biases_3_node)
    return layer3


In [18]:
y_train_prediction = network(X_train_node)
y_test_prediction = network(X_test_node)


In [19]:
cross_entropy = tf.losses.softmax_cross_entropy(y_train_node, y_train_prediction)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [20]:
optimizer = tf.train.AdamOptimizer(0.005).minimize(cross_entropy)


In [21]:
def calculate_accuracy(actual, predicted):
    actual = np.argmax(actual, 1)
    predicted = np.argmax(predicted, 1)
    return (100 * np.sum(np.equal(predicted, actual)) / predicted.shape[0])


In [23]:
import time

num_epochs = 100

with tf.Session() as session:
    tf.global_variables_initializer().run()
    for epoch in range(num_epochs):

        start_time = time.time()

        _, cross_entropy_score = session.run([optimizer, cross_entropy],
                                             feed_dict={X_train_node: raw_X_train, y_train_node: raw_y_train})

        if epoch % 10 == 0:
            timer = time.time() - start_time

            print('Epoch: {}'.format(epoch), 'Current loss: {0:.4f}'.format(cross_entropy_score),
                  'Elapsed time: {0:.2f} seconds'.format(timer))

            final_y_test = y_test_node.eval()
            final_y_test_prediction = y_test_prediction.eval()
            final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
            print("Current accuracy: {0:.2f}%".format(final_accuracy))

    final_y_test = y_test_node.eval()
    final_y_test_prediction = y_test_prediction.eval()
    final_accuracy = calculate_accuracy(final_y_test, final_y_test_prediction)
    print("Final accuracy: {0:.2f}%".format(final_accuracy))

final_fraud_y_test = final_y_test[final_y_test[:, 1] == 1]
final_fraud_y_test_prediction = final_y_test_prediction[final_y_test[:, 1] == 1]
final_fraud_accuracy = calculate_accuracy(final_fraud_y_test, final_fraud_y_test_prediction)
print('Final fraud specific accuracy: {0:.2f}%'.format(final_fraud_accuracy))

Epoch: 0 Current loss: 1.3983 Elapsed time: 11.34 seconds
Current accuracy: 0.16%
Epoch: 10 Current loss: 1.3973 Elapsed time: 8.44 seconds
Current accuracy: 31.79%
Epoch: 20 Current loss: 1.3760 Elapsed time: 10.57 seconds
Current accuracy: 7.90%
Epoch: 30 Current loss: 1.2771 Elapsed time: 11.06 seconds
Current accuracy: 67.31%
Epoch: 40 Current loss: 1.1020 Elapsed time: 10.28 seconds
Current accuracy: 96.76%
Epoch: 50 Current loss: 0.9816 Elapsed time: 10.06 seconds
Current accuracy: 98.73%
Epoch: 60 Current loss: 0.9070 Elapsed time: 9.13 seconds
Current accuracy: 98.54%
Epoch: 70 Current loss: 0.8739 Elapsed time: 10.79 seconds
Current accuracy: 99.39%
Epoch: 80 Current loss: 0.8574 Elapsed time: 9.28 seconds
Current accuracy: 99.32%
Epoch: 90 Current loss: 0.8449 Elapsed time: 10.92 seconds
Current accuracy: 99.55%
Final accuracy: 99.40%
Final fraud specific accuracy: 80.22%
